## 注意(attention)！开始做前必读项！
所有的代码一定要在这个文件里面编写，不要自己创建一个新的文件 对于提供的数据集，不要改存储地方，也不要修改文件名和内容 不要重新定义函数（如果我们已经定义好的），按照里面的思路来编写。当然，除了我们定义的部分，如有需要可以自行定义函数或者模块 写完之后，重新看一下哪一部分比较慢，然后试图去优化。一个好的习惯是每写一部分就思考这部分代码的时间复杂度和空间复杂度，AI工程是的日常习惯！ 
这次作业很重要，一定要完成！ 相信会有很多的收获！

In [2]:
# -*- coding: utf-8 -*-  

import numpy as np
import pandas as pd
from gensim import models
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.externals import joblib

In [3]:
data_path = '~/dataset/图书分类文本数据集/5813f3b1-617d-47ad-87e8-37a773a983af/file/归档/'

## 加载训练集文件和测试集文件

In [4]:
data_text,data=[],[]
stopWords=[]

! ls -l

def load_data():
    '''
    函数说明：该函数用于加载数据集
    return: 
        -data: 表示所有数据拼接的原始数据
        -data_text: 表示数据集中的特征数据集
        -datatext: 表示经过分词之后的特征数据集
        -stopWords: 表示读取的停用词
    '''
    print('load Pre_process')
    data = pd.concat([
        pd.read_csv(data_path + 'train_clean.csv', sep='\t'),
        pd.read_csv(data_path + 'dev_clean.csv', sep='\t'),
        pd.read_csv(data_path + 'test_clean.csv', sep='\t')
        ])
    print("读取数据集完成")
    
    data_text = list(data.text)  # .apply(lambda x: x.split(' '))
    datatext = []
    for i in range(len(data_text)):
        datatext.append(data_text[i].split(' '))
        
    stopWords = [line.strip() for line in open('stopwords.txt').readlines()]
    print("取停用词完成")
    return data, data_text,datatext, stopWords


total 116
-rw-r--r-- 1 root         root   2012 Jun 23 15:54 Dataset-图书分类文本数据集.ipynb
-rw-r--r-- 1 user10000440 users 21389 Jun 28 18:00 embedding(To_Do).ipynb
-rw-r--r-- 1 user10000440 users 83861 Jun 28 16:39 LightGBM_feature_engineering_grid(To_Do).ipynb
drwxr-xr-x 6 user10000440 users  4096 Jun 27 21:46 output
drwxr-xr-x 4 user10000440 users  4096 Jun 24 18:17 src
-rw-r--r-- 1 user10000440 users  5273 Jun 23 17:59 stopwords.txt
-rw-r--r-- 1 user10000440 users    72 Jun 25 21:22 Untitled1.ipynb
-rw-r--r-- 1 user10000440 users   555 Jun 26 12:58 Untitled2.ipynb
-rw-r--r-- 1 user10000440 users  3312 Jun 27 21:42 Untitled.ipynb


In [5]:
data, data_text, datatext, stopWords=load_data()

load Pre_process
读取数据集完成
取停用词完成


In [ ]:
def trainer_tfidf():
    '''
    函数说明：该函数用于训练tfidf的词向量
    return: 
        -tfidf: 表示经过TF-ID模型训练出的词向量
    '''
    # ToDo
    # 根据数据集训练tfidf的词向量
    # 第一步：首先通过 TfidfVectorizer 创建一个模型对象
    count_vect = TfidfVectorizer(stop_words=stopWords , max_df=0.6, ngram_range=(1, 2))
    # 第二步：用模型对象去fit训练数据集
    tfidf = count_vect.fit(data_text)
    print('train tfidf_embedding')
    #返回是一个稀疏矩阵
    return tfidf


In [ ]:
tfidf = trainer_tfidf()

根据数据集训练word2vec的词向量  
利用models.Word2Vec训练w2v的词向量  
部分参数说明：  
min_count：表示低于该频率的词将会删除，  
window：表示滑动窗口大小，  
alpha：表示学习率，  
negative：表示负采样样本个数，  
max_vocab_size：表示RAM中最大限制的词个数  

In [ ]:
def trainer_w2v():
    '''
    函数说明：该函数基于Word2vec模型训练词向量
    return: 
        -w2v: 表示经过word2vec模型训练出的词向量
    '''
    print('train word2vec Embedding')
    # 训练w2v的词向量
    # 第一步：利用models.Word2Vec构建一个模型，
    w2v = models.Word2Vec(min_count=2,
                                window=3,
                                size=300,
                                sample=6e-5,
                                alpha=0.03,
                                min_alpha=0.0007,
                                negative=15,
                                workers=4,
                                iter=10,
                                max_vocab_size=50000) 
        
    w2v.build_vocab(datatext)

    w2v.train(data_text,
              total_examples=w2v.corpus_count,
              epochs=15,
              report_delay=1)
        
    print('train fast_embedding')
    return w2v

In [ ]:
w2v = trainer_w2v()

In [6]:
def trainer_fasttext():
    '''
    函数说明：该函数基于FastText模型训练词向量
    return: 
        -fast: 表示经过FastText模型训练出的词向量
    '''
    # ToDo
    # 根据数据集训练FastTExt的词向量
    # hint: 利用models.FastText,
    # 可使用部分参数说明：
    # size：生成的向量维度，
    # window: 移动窗口，
    # aphla: 学习率，
    # min_count: 对低于该频率的词进行截断
    # 可以参照trainer_w2v函数完成FastText的词向量的训练
    # 可以直接根据models.FastText直接训练样本获取词向量
    fast = models.FastText(size=300, window=3, alpha=0.05, min_count=2)  # instantiate
    fast.build_vocab(sentences=datatext)
    fast.train(sentences=data_text, total_examples=fast.corpus_count, epochs=10)  # train
    return fast

In [7]:
fast= trainer_fasttext()

In [8]:
print(len(fast.wv.vocab.keys()))

217943


In [9]:
root_path = '/home/user10000440/notespace'
def saver():
    '''
    函数说明：该函数存储训练好的模型
    '''
    #Todo
    # hint: 通过joblib.dump保存tfidf
#     print('save tfidf model')
#     joblib.dump(tfidf, root_path +'/src/Embedding/models/tfidf_model')

#     # hint: w2v可以通过自带的save函数进行保存
#     print('save word2vec model')
#     w2v.save(root_path + '/src/Embedding/models/w2v_model.w2v')
#     w2v.wv.save_word2vec_format(root_path + '/src/Embedding/models/w2v_model_wv.bin',binary=True)

     # hint: fast可以通过自带的save函数进行保存
    print('save fast model')
    fast.save(root_path + '/src/Embedding/models/fasttext_model')
    fast.wv.save_word2vec_format(root_path + '/src/Embedding/models/fasttext_model_wv.bin',binary=True)
    
saver()


save fast model


In [ ]:

def load():
    '''
    函数说明：该函数加载训练好的模型
    '''
    #ToDo
    # 加载模型 
    # hint: tfidf可以通过joblib.load进行加载
    # w2v和fast可以通过gensim.models.KeyedVectors.load加载
    print('load tfidf_embedding model')
    tfidf = joblib.load(filename=root_path +'/src/Embedding/models/tfidf_model')

    print('load w2v_embedding model')
    #w2v = gensim.models.word2vec.Word2Vec.load(root_path + '/src/Embedding/models/w2v_model.w2v')
    w2v = models.KeyedVectors.load(root_path + '/src/Embedding/models/w2v_model.w2v')
    w2v_wv = models.KeyedVectors.load_word2vec_format(root_path + '/src/Embedding/models/w2v_model.w2v', binary=True)

    print('load fast_embedding model')
    fast = models.KeyedVectors.load(root_path + '/src/Embedding/models/fasttext_model')
    
    print('load end')
    
load()